In [1]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 
              'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 
              'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':str, 'condition':int, 
              'lat':float, 'date':str, 
              'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

In [3]:
df_train = pd.read_csv('kc_house_train_data.csv', dtype=dtype_dict)
df_train.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3.0,1.00,1180.0,5650,1,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340.0,5650.0
1,6414100192,20141209T000000,538000.0,3.0,2.25,2570.0,7242,2,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690.0,7639.0
2,5631500400,20150225T000000,180000.0,2.0,1.00,770.0,10000,1,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720.0,8062.0
3,2487200875,20141209T000000,604000.0,4.0,3.00,1960.0,5000,1,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360.0,5000.0
4,1954400510,20150218T000000,510000.0,3.0,2.00,1680.0,8080,1,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800.0,7503.0


In [4]:
df_test = pd.read_csv('kc_house_test_data.csv', dtype=dtype_dict)
df_test.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,0114101516,20140528T000000,310000.0,3.0,1.0,1430.0,19901,1.5,0,0,...,7,1430,0,1927,0,98028,47.7558,-122.229,1780.0,12697.0
1,9297300055,20150124T000000,650000.0,4.0,3.0,2950.0,5000,2,0,3,...,9,1980,970,1979,0,98126,47.5714,-122.375,2140.0,4000.0
2,1202000200,20141103T000000,233000.0,3.0,2.0,1710.0,4697,1.5,0,0,...,6,1710,0,1941,0,98002,47.3048,-122.218,1030.0,4705.0
3,8562750320,20141110T000000,580500.0,3.0,2.5,2320.0,3980,2,0,0,...,8,2320,0,2003,0,98027,47.5391,-122.070,2580.0,3980.0
4,7589200193,20141110T000000,535000.0,3.0,1.0,1090.0,3000,1.5,0,0,...,8,1090,0,1929,0,98117,47.6889,-122.375,1570.0,5080.0


In [5]:
df_train['bedrooms_squared'] = df_train['bedrooms']*df_train['bedrooms']
df_train['bed_bath_rooms'] = df_train['bedrooms'] * df_train['bathrooms']
df_train['log_sqft_living'] = pd.np.log(df_train['sqft_living'])
df_train['lat_plus_long'] = df_train['lat'] + df_train['long']

In [6]:
df_test['bedrooms_squared'] = df_test['bedrooms']*df_test['bedrooms']
df_test['bed_bath_rooms'] = df_test['bedrooms'] * df_test['bathrooms']
df_test['log_sqft_living'] = pd.np.log(df_test['sqft_living'])
df_test['lat_plus_long'] = df_test['lat'] + df_test['long']

<hr>
4. Quiz Question: what are the mean (arithmetic average) values of your 4 new variables on TEST data? (round to 2 digits)

In [7]:
bedrooms_squared_mean = df_test['bedrooms_squared'].mean()
bed_bath_rooms_mean = df_test['bed_bath_rooms'].mean()
log_sqft_living_mean = df_test['log_sqft_living'].mean()
lat_plus_long_mean = df_test['lat_plus_long'].mean()

print('''
bedrooms_squared = {:.2f}
bed_bath_rooms = {:.2f}
log_sqft_living = {:.2f}
lat_plus_long = {:.2f}'''
      .format(bedrooms_squared_mean,bed_bath_rooms_mean, log_sqft_living_mean, lat_plus_long_mean))


bedrooms_squared = 12.45
bed_bath_rooms = 7.50
log_sqft_living = 7.55
lat_plus_long = -74.65


<hr>

1. Model 1: ‘sqft_living’, ‘bedrooms’, ‘bathrooms’, ‘lat’, and ‘long’

1. Model 2: ‘sqft_living’, ‘bedrooms’, ‘bathrooms’, ‘lat’,‘long’, and ‘bed_bath_rooms’

1. Model 3: ‘sqft_living’, ‘bedrooms’, ‘bathrooms’, ‘lat’,‘long’, ‘bed_bath_rooms’, ‘bedrooms_squared’, ‘log_sqft_living’, and ‘lat_plus_long’

In [8]:
def compute_model(df, features, label):
    h = df[features]
    hTh = h.T.values.dot(h.values)
    hTh_inv = pd.np.linalg.inv(hTh)
    hTh_inv_hT = hTh_inv.dot(h.T.values)

    return hTh_inv_hT.dot(df[label])

In [9]:
m1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']

w1 = compute_model(df_train, m1_features, 'price')

In [10]:
m2_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long', 'bed_bath_rooms']

w2 = compute_model(df_train, m2_features, 'price')

In [11]:
m3_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long', 'bed_bath_rooms', 'bedrooms_squared', 'log_sqft_living',  'lat_plus_long']

w3 = compute_model(df_train, m3_features, 'price')

<hr>
6. Quiz Question: What is the sign (positive or negative) for the coefficient/weight for ‘bathrooms’ in Model 1?

In [12]:
print('the sign of the weight for bathrooms in Model 1 is {}'.format(pd.np.sign(w1[2])))

the sign of the weight for bathrooms in Model 1 is 1.0


7. Quiz Question: What is the sign (positive or negative) for the coefficient/weight for ‘bathrooms’ in Model 2?

In [13]:
print('the sign of the weight for bathrooms in Model 2 is {}'.format(pd.np.sign(w2[2])))

the sign of the weight for bathrooms in Model 2 is -1.0


In [14]:
w1, w2, w3

(array([ 3.00963659e+02, -5.95548880e+04,  5.32107730e+03,  5.32596984e+05,
         2.06418680e+05]),
 array([ 2.93999642e+02, -1.30492300e+05, -1.09046396e+05,  5.32930482e+05,
         2.04561518e+05,  3.36896733e+04]),
 array([ 1.12372780e+02,  7.08401431e+04,  4.87027814e+05,  2.16897513e+05,
        -1.49541062e+05, -1.14542918e+04, -2.30920367e+03, -6.33873612e+05,
         3.28648158e+05]))

<hr>
10. Quiz Question: Which model (1, 2 or 3) had the lowest RSS on TRAINING data?

In [15]:
def get_RSS(df, features, label, weights):
    h = df[features]
    y = df[label]
    
    y_hw = y - h.values.dot(weights)
    
    return y_hw.T.values.dot(y_hw.values)

In [16]:
r1 = get_RSS(df_train, m1_features, 'price', w1)
r2 = get_RSS(df_train, m2_features, 'price', w2)
r3 = get_RSS(df_train, m3_features, 'price', w3)

In [17]:
r1, r2, r3

(1065848138856088.8, 1049264468667266.5, 2364861972630846.0)

In [18]:
pd.np.min([r1, r2, r3])

1049264468667266.5

todo: This answer is wrong, the quiz accepted 3rd model.

<hr>

12. Quiz Question: Which model (1, 2, or 3) had the lowest RSS on TESTING data?

In [19]:
test_r1 = get_RSS(df_test, m1_features, 'price', compute_model(df_test, m1_features, 'price'))
test_r2 = get_RSS(df_test, m2_features, 'price', compute_model(df_test, m2_features, 'price'))
test_r3 = get_RSS(df_test, m3_features, 'price', compute_model(df_test, m3_features, 'price'))

In [20]:
test_r1, test_r2, test_r3

(248057921575609.66, 243338379094892.94, 1.0019740247208374e+16)

In [21]:
pd.np.min([test_r1, test_r2, test_r3])

243338379094892.94